# Processing Pbtd01-03.czi

In [24]:
# Import my packages.
import flymovie as fm
import os
import copy
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi

# Activate package autoload.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
%reload_ext autoreload

## 1. Read input files

In [16]:
# Set channels corresponding to nuclear signal and MS2 spots.
spotchannel = 0

surface_before = 2491
surface_after = 2491.1

# Set files for output movie object and input files.
czi_folder = '/Users/michaelstadler/Downloads/'

czi_files = ['Pbtd01-03.czi'
]

czi_file = '/Users/michaelstadler/Downloads/Pbtd01-02.czi'

In [17]:
stack, starting_positions, z_interval = fm.read_czi(czi_file, return_metadata=True, swapaxes=False)
join_frame = []

In [35]:
stack = stack[:,34:]

In [36]:
fm.viewer(stack, 10, zmax=True)

interactive(children=(Dropdown(description='Color', index=2, options=('Reds', 'gray_r', 'cividis', 'prism', 'g…

## 2. Nuclear Segmentation
### 2-1 Determine parameters for segmentation of nuclei.

In [ ]:
# Perform initial nuclear segmentation using the first few frames to determine parameters.
min_nuc_center_dist = 40
sigma = 5
usemax = True

nucmask_test = fm.segment_nuclei_4dstack(stack[1, :1], fm.segment_nuclei.segment_nuclei_3Dstack_rpb1, min_nuc_center_dist=min_nuc_center_dist, sigma=sigma, usemax=usemax, display=False)
fm.viewer(nucmask_test, 10, color='prism')

### 2-2. Perform initial segmentation

In [ ]:
# Segment nuclei using parameters determined above.
nucmask_init = fm.segment_nuclei_4dstack(stack[nucchannel], fm.segment_nuclei_3Dstack_rpb1, min_nuc_center_dist=min_nuc_center_dist, sigma=sigma, usemax=usemax)
fm.viewer(nucmask_init, figsize=10, color='prism')

### 2-3. Filter nuclear mask for object size.

In [ ]:
# Filter segmented nuclei based on their size.
size_min = 0
size_max = 1.1e5

nucmask_sizefiltered = fm.labelmask_filter_objsize_apply4d(nucmask_init, size_min, size_max)
fm.viewer(nucmask_sizefiltered, figsize=10, color="viridis", init_maxval=1)

### 2-4 (optional) Filter nuclei based on circularity

In [ ]:
# Filter segmented nuclei based on circularity (0-1).
slicenum = 6 # Z slice to use for circularity -- irrelevant if using max projections.
circularity_min = 0.5

nucmask_filtercirc = fm.filter_labelmask_circularity_apply4d(nucmask_sizefiltered, slicenum=slicenum, 
    circularity_min=circularity_min)

fm.viewer(nucmask_filtercirc, color="viridis", init_maxval=1)

### Select nuclear mask to use

In [ ]:
nucmask_to_connect = nucmask_sizefiltered
#nucmask_to_connect = nucmask_filtercirc

### 2-5 Connect nuclei between frames

In [ ]:
max_frames_skipped = 2

nucmask_conn = fm.connect_nuclei(nucmask_to_connect, max_frames_skipped=max_frames_skipped, usemax=True)
fm.viewer(nucmask_conn, 10, color='prism')

### 2-6 Interpolate to fill in gaps (optional)

In [ ]:
#nucmask = fm.interpolate_nuclear_mask(nucmask_conn, 3, usemax=True)
nucmask = nucmask_conn

#### View final mask

In [ ]:
fm.viewer(nucmask, color="viridis", init_maxval=1)

#### Clean up intermediate objects

In [ ]:
del nucmask_init, nucmask_sizefiltered, 
try: 
    del nucmask_filtercirc
except:
    pass

In [19]:
nucmask = np.zeros_like(stack)
stack = np.expand_dims(stack, 0)

## 3. Detecting MS2 spots
### 3-1 Perform gaussian fitting on MS2 channel

In [177]:
min_distances = (stack.shape[-2], 5, 5)
fitwindow_rad_xy = 10
fitwindow_rad_z = 2
sigma_small = 1

fits = fm.fit_ms2(stack[spotchannel], min_distances = min_distances, fitwindow_rad_xy=fitwindow_rad_xy, sigma_small=sigma_small, mode='mindist', use_thresh=True, nucmask=nucmask)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 

#### View fits

In [183]:
fm.quickview_ms2(stack, fits, 0, figsize=10)

interactive(children=(Dropdown(description='Color', index=2, options=('Reds', 'gray_r', 'cividis', 'prism', 'g…

### 3-2 Filter fits

In [184]:
# Parameters for filtering fit
peakiness = 4.3
fits_filtered = fm.filter_ms2fits(fits, peakiness, stack, spotchannel)
fm.quickview_ms2(stack, fits_filtered, 0)

interactive(children=(Dropdown(description='Color', index=2, options=('Reds', 'gray_r', 'cividis', 'prism', 'g…

### 3-3 Connect spots between frames

In [245]:
#spots_connected = fm.connect_ms2_frames_via_nuclei(fits_filtered, nucmask, nucmask_dilation=7)
reload(fm)
spots_connected = fm.connect_ms2_frames_nearest(fits_filtered, nucmask, max_jump=30, max_frame_gap=10)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 

In [246]:
fm.quickview_ms2(stack, spots_connected, MAX=True, halfwidth_z=0)

interactive(children=(Dropdown(description='Color', index=2, options=('Reds', 'gray_r', 'cividis', 'prism', 'g…

In [205]:
fits[12][fits[12][:,0] == 14]

array([[1.40000000e+01, 9.00000000e+01, 6.07000000e+02, 3.48212958e+02,
        3.64333042e+04, 8.65379130e+00, 2.51305525e+01],
       [1.40000000e+01, 1.66000000e+02, 6.52000000e+02, 8.48284761e+02,
        2.61297819e+00, 6.12220044e+00, 5.14562711e+00],
       [1.40000000e+01, 1.62000000e+02, 4.05000000e+02, 2.14704654e+03,
        2.04118657e+00, 3.90472045e+00, 3.44820633e+00],
       [1.40000000e+01, 2.07000000e+02, 5.69000000e+02, 2.14008713e+03,
        1.82353004e+00, 3.65090535e+00, 3.45818172e+00],
       [1.40000000e+01, 2.17000000e+02, 5.11000000e+02, 3.11119480e+03,
        1.62402366e+00, 3.63930310e+00, 3.20357104e+00]])

In [202]:
spots_connected[33]

array([[8.00000000e+00, 0.00000000e+00, 1.30000000e+01, 2.08000000e+02,
        5.71000000e+02, 5.18727490e+02, 4.82778108e+00, 6.80250561e+00,
        7.05111784e+00],
       [9.00000000e+00, 0.00000000e+00, 1.20000000e+01, 1.96000000e+02,
        5.71000000e+02, 1.40457487e+03, 1.69323291e+00, 3.52465980e+00,
        3.63347323e+00],
       [1.00000000e+01, 0.00000000e+00, 1.30000000e+01, 1.99000000e+02,
        5.72000000e+02, 1.27321185e+03, 2.10387002e+00, 3.84344984e+00,
        4.03278035e+00],
       [1.10000000e+01, 0.00000000e+00, 1.20000000e+01, 2.08000000e+02,
        5.71000000e+02, 1.28871257e+03, 2.77713680e+00, 4.61130552e+00,
        3.86417071e+00],
       [1.20000000e+01, 0.00000000e+00, 1.40000000e+01, 2.07000000e+02,
        5.69000000e+02, 2.14008713e+03, 1.82353004e+00, 3.65090535e+00,
        3.45818172e+00],
       [1.30000000e+01, 0.00000000e+00, 1.40000000e+01, 2.06000000e+02,
        5.66000000e+02, 2.03579934e+03, 3.09266578e+00, 3.81697092e+00,
        3.0

In [201]:
spots_connected[56]

array([[1.20000000e+01, 0.00000000e+00, 1.40000000e+01, 2.07000000e+02,
        5.69000000e+02, 2.14008713e+03, 1.82353004e+00, 3.65090535e+00,
        3.45818172e+00]])

### 3-4 Filter detected spots on trajectory length

In [ ]:
min_len = 1
spot_data_init = fm.filter_spot_duration(spots_connected, min_len)

### 3-5 Review spot calling

#### Record bad spot calls and missing spot calls

In [ ]:
fm.quickview_ms2(stack, spot_data_init, channel = spotchannel, MAX=True, grid=False)

In [ ]:
missing_spots = (
    # spot_id, frame_number
    (28,0),
    (6,1),
    (27,2),
    (27,3),
    (3,6),
    (21,8),
    (21,9),
    (23,8),
    (6,16),
    (13,19),
    (27,19),
    (17,21),
    (42,22),
    (26,22),
    (27,28),
    (27,30)
)

bad_spots = []


### 3-6 Manually enter coordinates for missing spots

In [ ]:
new_spots = []
fm.add_missing_spots(spot_data_init, stack, missing_spots, new_spots, channel=spotchannel)

### 3-7 Apply manual corrections

In [ ]:
spot_data = fm.spot_data_apply_manual_curations(spot_data_init, new_spots, bad_spots, 
    stack, fitwindow_rad_xy=fitwindow_rad_xy, fitwindow_rad_z=fitwindow_rad_z, channel=spotchannel)

#### View final spot detections

In [ ]:
fm.quickview_ms2(stack, spot_data, channel=spotchannel)

## 4. Integrate signal around spots, apply corrections.


In [ ]:
paramgrids = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/20210616-depth-correction-params.pkl')

In [ ]:
mv = fm.movie(stack, nucmask, fits, spot_data)
fm.analyze.mv_process_apply_corrections(mv, paramgrids, surface_before, surface_after, join_frames, starting_positions, z_interval, ij_rad=7, z_rad=2, fit_depth_min=11, fit_depth_max=14)


## 5. QC

In [ ]:
# Plot some traces to see if they look reasonable.
fm.spotdf_plot_traces([mv.ms2_bleachcorr], minlen=30, sigma=0.05, norm=False, figsize=(16,10))

In [ ]:
fm.plot_projections(mv, channel=spotchannel)


In [ ]:
# View "spot movies", small boxes of stack data centered on detected spots.
spot_movies = fm.spot_movies(mv.stack, mv.spot_data, channel=spotchannel, len_ij=51, fill=0)
# Mean projection of z slices -1 to +1:
#fm.viewer(spot_movies[:,:,2:5].mean(axis=2), 5, order='ntxy', color='cividis')
# No projection (with Z axis):
fm.viewer(spot_movies, 5, order='ntzxy', color='cividis')

In [ ]:
# Optional: Examine individual spots in original data.
spot_to_view = 42
fm.quickview_ms2(mv.stack, mv.spot_data, channel=1, spot_id=spot_to_view, MAX=True, halfwidth_z=1)

### 6. Save movie

In [ ]:
fm.save_pickle(mv, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/ms2_mv_obects/20210621-rpb1egfp-eve-em2-02-09.pkl')